<center>

## ***Company-based Capstone Project***
#### **Team ID		: C24-MR01**

Team Member	: 

(ML) M002D4KY2877 - Auvarifqi Putra Diandra

(ML) M010D4KY3370 - Rafi Madani

(ML) M002D4KY2625 - Iskandar Muda Rizky Parlambang

(MD) A010D4KY4202 - Muhammad Adryan Haska Putra

(MD) A297D4KX4551 - Vena Feranica

(CC) C002D4KY1032 - Muhammad Naufal

(CC) C459D4KY0090 - Jamaludin Ahmad Rifai



---

In [1]:
from typing import Dict, Text
from pathlib import Path

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [9]:
import pandas as pd

data_path = './data'
ratings_df = pd.read_csv(f"{data_path}/ratings.csv")
ratings_df.head(10)

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211
1,25.0,4,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525
2,18.0,4,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439
3,50.0,5,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472
4,50.0,10,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094
5,18.0,7,325,Crash (1996),19.0,876346551,False,601,1,artist,4.0,99687
6,18.0,2,95,Aladdin (1992),19.0,882064434,True,710,17,student,3.0,92020
7,25.0,0,92,True Romance (1993),34.0,875135363,True,833,21,writer,2.0,90019
8,25.0,4,425,Bob Roberts (1992),27.0,880844102,True,916,18,engineer,5.0,N2L5N
9,25.0,0,271,Starship Troopers (1997),32.0,884801053,True,940,2,administrator,2.0,02215


In [10]:
ratings_df.info()
total_rows, total_attributes = ratings_df.shape
print()
print('Jumlah data:', total_rows)
print("Jumlah atribut:", total_attributes)

ratings_df = ratings_df.drop_duplicates()

ratings_df.info()
total_rows, total_attributes = ratings_df.shape
print()
print('Jumlah data:', total_rows)
print("Jumlah atribut:", total_attributes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   bucketized_user_age    100000 non-null  float64
 1   movie_genres           100000 non-null  int64  
 2   movie_id               100000 non-null  int64  
 3   movie_title            100000 non-null  object 
 4   raw_user_age           100000 non-null  float64
 5   timestamp              100000 non-null  int64  
 6   user_gender            100000 non-null  bool   
 7   user_id                100000 non-null  int64  
 8   user_occupation_label  100000 non-null  int64  
 9   user_occupation_text   100000 non-null  object 
 10  user_rating            100000 non-null  float64
 11  user_zip_code          100000 non-null  object 
dtypes: bool(1), float64(3), int64(5), object(3)
memory usage: 8.5+ MB

Jumlah data: 100000
Jumlah atribut: 12
<class 'pandas.core.frame.DataFrame'>

In [11]:
ratings_df.shape
np.random.seed(0)

In [12]:
ratings_df['like'] = np.random.randint(0, 2, size=len(ratings_df))

In [13]:
ratings_df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,like
0,45.0,7,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211,0
1,25.0,4,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525,1
2,18.0,4,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439,1
3,50.0,5,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472,0
4,50.0,10,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094,1


In [15]:
user_ids = ratings_df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = ratings_df["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
ratings_df["user"] = ratings_df["user_id"].map(user2user_encoded)
ratings_df["movie"] = ratings_df["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
ratings_df["user_rating"] = ratings_df["user_rating"].values.astype(np.float32)
ratings_df["like"] = ratings_df["like"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(ratings_df["user_rating"])
max_rating = max(ratings_df["user_rating"])
min_age = min(ratings_df["bucketized_user_age"])
max_age = max(ratings_df["bucketized_user_age"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 943, Number of Movies: 1682, Min rating: 1.0, Max rating: 5.0


In [17]:
df = ratings_df.sample(frac=1, random_state=42)
x = df[["user", "movie"]].values

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

df["rating_norm"] = df["user_rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
df["target"] = df.apply(lambda row: sigmoid(row["rating_norm"] + row["like"]), axis=1)
y = df["target"]

# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print(x, y)
print("Jumlah data train: ", len(x_train))
print("Jumlah data val: ", len(x_val))

[[904 998]
 [485 647]
 [134 388]
 ...
 [394 162]
 [176 508]
 [171 477]] 75721    0.851953
80184    0.851953
19864    0.777300
76699    0.731059
92991    0.817574
           ...   
6265     0.880797
54886    0.817574
76820    0.817574
860      0.679179
15795    0.562177
Name: target, Length: 100000, dtype: float64
Jumlah data train:  80000
Jumlah data val:  20000


In [22]:
from tensorflow import keras
from keras import layers

EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # rating_output = tf.nn.sigmoid(x)
        # like_output = tf.nn.sigmoid(x)
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.build(input_shape=(None, 2))
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    # metrics=[tf.keras.metrics.RootMeanSquaredError()]
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()

print(f'Num users: {num_users}, Num movies: {num_movies}')
print(f'Embedding size: {EMBEDDING_SIZE}')
print(model.user_embedding)
print(model.movie_embedding)
print(model.user_bias)
print(model.movie_bias)

C:\Users\Iskandar\Documents\GitHub\nbs-ml\venv-collab\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'recommender_net_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "recommender_net_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Num users: 943, Num movies: 1682
Embedding size: 50
<Embedding name=embedding_4, built=False>
<Embedding name=embedding_6, built=False>
<Embedding name=embedding_5, built=False>
<Embedding name=embedding_7, built=False>


In [23]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=15,
    verbose=1,
    validation_data=(x_val, y_val),
)

Epoch 1/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6933 - root_mean_squared_error: 0.2638 - val_loss: 0.6936 - val_root_mean_squared_error: 0.2641
Epoch 2/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6898 - root_mean_squared_error: 0.2602 - val_loss: 0.6918 - val_root_mean_squared_error: 0.2624
Epoch 3/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6861 - root_mean_squared_error: 0.2568 - val_loss: 0.6900 - val_root_mean_squared_error: 0.2607
Epoch 4/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6819 - root_mean_squared_error: 0.2525 - val_loss: 0.6881 - val_root_mean_squared_error: 0.2589
Epoch 5/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6789 - root_mean_squared_error: 0.2500 - val_loss: 0.6862 - val_root_mean_squared_error: 0.2571
Epoch 6/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6751 - root_mean_squared_error: 0.2464 - val_loss: 0.6843 - val_root_mean_squared_error: 0.2552
Epoch 7/15
1250/1250 ━━━━━━━━━━━━━━━━━━━

In [26]:
# Mendapatkan data frame dengan movieId unik
movie_df = ratings_df[['movie_id']].drop_duplicates()

# Memilih satu user secara acak
user_id = df.user_id.sample(1).iloc[0]
movies_watched_by_user = df[df.user_id == user_id]
movies_not_watched = movie_df[
    ~movie_df["movie_id"].isin(movies_watched_by_user.movie_id.values)
]["movie_id"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)

ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="user_rating", ascending=False)
    .head(5)
    .movie_id.values
)
movie_df_rows = movie_df[movie_df["movie_id"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movie_id"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Showing recommendations for user: 49
Movies with high ratings from user
--------------------------------
433 : Heathers (1989)
175 : Brazil (1985)
428 : Harold and Maude (1971)
209 : This Is Spinal Tap (1984)
47 : Ed Wood (1994)
--------------------------------
Top 10 movie recommendations
--------------------------------
265 : Hunt for Red October, The (1990)
79 : Fugitive, The (1993)
89 : Blade Runner (1982)
64 : Shawshank Redemption, The (1994)
274 : Sabrina (1995)
97 : Dances with Wolves (1990)
81 : Hudsucker Proxy, The (1994)
109 : Mystery Science Theater 3000: The Movie (1996)
66 : While You Were Sleeping (1995)
83 : Much Ado About Nothing (1993)
